In [4]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='postgres',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = ''' CREATE TABLE IF NOT EXISTS Expresso_2023 (
                                    DATA_CRIACAO      text,
                                    CENTRO    text,
                                    Giro text,
                                    LOPTICA  text,
                                    JANELA_HORARIA text,
                                    NOME text,
                                    MORADA text,
                                    CP text,
                                    LOCALIDADE text,
                                    COD_T_EVEN text,
                                    DATA_EVENTO text,
                                    LATITUDE text,
                                    LONGITUDE text) '''
criar_db(sql)


In [5]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor()
cur.execute("SELECT DISTINCT CONCAT(SUBSTRING(DATA_CRIACAO, 7, 4),SUBSTRING(DATA_CRIACAO, 4, 2),SUBSTRING(DATA_CRIACAO, 1, 2)) AS ConvertedDate FROM Expresso_2023")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

['20221231' '20230102' '20230103' '20230104' '20230105' '20230106'
 '20230107' '20230109' '20230110' '20230111' '20230112' '20230113'
 '20230114' '20230116' '20230117' '20230118' '20230119' '20230120'
 '20230121' '20230123' '20230124' '20230125' '20230126' '20230127'
 '20230128' '20230130' '20230131' '20230201' '20230202' '20230203'
 '20230204' '20230206' '20230207' '20230208' '20230209' '20230210'
 '20230211' '20230213' '20230214' '20230215' '20230216' '20230217'
 '20230218' '20230220' '20230222' '20230223' '20230224' '20230225'
 '20230227' '20230228' '20230301' '20230302' '20230303' '20230304'
 '20230306' '20230307' '20230308' '20230309' '20230310' '20230311'
 '20230313' '20230314' '20230315' '20230316' '20230317' '20230318'
 '20230320' '20230321' '20230322' '20230323' '20230324' '20230327'
 '20230328' '20230329' '20230330' '20230331' '20230401' '20230403'
 '20230404' '20230405' '20230406' '20230410' '20230411' '20230412'
 '20230413' '20230414' '20230417' '20230418' '20230419' '20230

Expresso

In [6]:
from ftplib import FTP
import os
import io
import datetime
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import time

current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/EventosPDA')

files = ftp.nlst()

def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')
    return decoded_line.strip()



data_validation = []


cur = con.cursor()

cur.execute("SELECT distinct  SUBSTRING(DATA_CRIACAO, 7, 4) ||  SUBSTRING(DATA_CRIACAO, 4, 2) ||  SUBSTRING(DATA_CRIACAO, 1, 2) as dias FROM Expresso_2023 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

cur.close()

for file in files:
    start_time = time.time()
    if file.startswith("E"):
        date_str_s = file[14:22].replace("/", "")
        date_str_s_array = np.array(date_str_s)
        if date_str_s not in data_validation:
            
            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/EventosPDA')
            
            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            # Process the data line by line, excluding the last line
            lines = file_data.readlines()
            data_rows = [process_data(row) for row in lines[:-1]]  # Exclude the last line
            df = pd.read_csv(io.StringIO('\n'.join(data_rows)), sep=';')
            ftp.quit()
            
            if len(data_rows)==1: 
                continue
            else:
                # Create a DataFrame from the cleaned data rows


                # Insert the data into the database
                for index, row in df.iterrows():
                    try:
                        cur = con.cursor()
                        # Replace "NaN" values with None (NULL) for database insertion
                        row = [None if pd.isna(value) else value for value in row]
                        placeholders = ', '.join(['%s'] * len(row))
                        insert_query = f"INSERT INTO Expresso_2023 ({', '.join(df.columns)}) VALUES ({placeholders})"
                        cur.execute(insert_query, tuple(row))
                    except (Exception, psycopg2.DatabaseError) as error:
                        print(f"Error inserting row {index+1}: {error}")
                        con.rollback()
                    else:
                        con.commit()
                        
                end_time = time.time()    
                print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")

con.close()




File 'EVPDA3466_ENV_20230902_20230904000025.CSV' inserted into the database: 0.6833021640777588 seconds


BANCA 

In [7]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='postgres',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = ''' CREATE TABLE IF NOT EXISTS BANCA_2023 (
                                    COD_CLIENTE_BNC text,
                                    CODIGO_BALCAO_SIDIR text,
                                    CODIGO_BALCAO_AF text,
                                    AGENCIA text,
                                    CA text,
                                    CIRCUITO text,
                                    DATA text,
                                    CONTACTO text,
                                    FATURAR text,
                                    ENTREGUE text,
                                    TAREFA text,
                                    CP text,
                                    CENTRO_OPERACIONAL text,
                                    MORADA_TOQUE text,
                                    HORA_TOQUE_PREVISTA text,
                                    HORA_TOQUE_REAL text
                                    ) '''
criar_db(sql)





In [8]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute("SELECT distinct REPLACE(data, '.', '') dias FROM Banca_2023 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

['20230102' '20230103' '20230104' '20230105' '20230106' '20230109'
 '20230110' '20230111' '20230112' '20230113' '20230116' '20230117'
 '20230118' '20230119' '20230120' '20230123' '20230124' '20230125'
 '20230126' '20230127' '20230130' '20230131' '20230201' '20230202'
 '20230203' '20230206' '20230207' '20230208' '20230209' '20230210'
 '20230213' '20230214' '20230215' '20230216' '20230217' '20230220'
 '20230221' '20230222' '20230223' '20230224' '20230227' '20230228'
 '20230301' '20230302' '20230303' '20230306' '20230307' '20230308'
 '20230309' '20230310' '20230313' '20230314' '20230315' '20230316'
 '20230317' '20230320' '20230321' '20230322' '20230323' '20230324'
 '20230327' '20230328' '20230329' '20230330' '20230331' '20230403'
 '20230404' '20230405' '20230406' '20230407' '20230410' '20230411'
 '20230412' '20230413' '20230414' '20230417' '20230418' '20230419'
 '20230420' '20230421' '20230424' '20230425' '20230426' '20230427'
 '20230428' '20230501' '20230502' '20230503' '20230504' '20230

In [9]:
from ftplib import FTP
import os
import io
import datetime
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import time

current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/Banca_Recolhas')

files = ftp.nlst()

def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')
    return decoded_line.strip()

data_validation = []

con = psycopg2.connect(host='localhost', 
                    database='postgres',
                    user='postgres', 
                    password='postgres')

cur = con.cursor()

cur.execute("SELECT distinct REPLACE(data, '.', '') dias FROM Banca_2023 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

cur.close()

# Create a mapping between input column names and database column names
column_mapping = {
    'ï»¿COD_CLIENTE_BNC': 'COD_CLIENTE_BNC',
    'CODIGO_BALCAO_SIDIR': 'CODIGO_BALCAO_SIDIR',
    'CODIGO_BALCAO_AF': 'CODIGO_BALCAO_AF',
    'AGENCIA': 'AGENCIA',
    'CA': 'CA',
    'CIRCUITO': 'CIRCUITO',
    'DATA': 'DATA',
    'CONTACTO': 'CONTACTO',
    'FATURAR': 'FATURAR',
    'ENTREGUE': 'ENTREGUE',
    'TAREFA': 'TAREFA',
    'CP': 'CP',
    'CENTRO OPERACIONAL': 'CENTRO_OPERACIONAL',
    'MORADA TOQUE': 'MORADA_TOQUE',
    'HORA TOQUE PREVISTA': 'HORA_TOQUE_PREVISTA',
    'HORA TOQUE REAL': 'HORA_TOQUE_REAL'
}

for file in files:
    start_time = time.time()
    if file.startswith("C"):
        date_str_s = file[21:29].replace("/", "")
        date_str_s_array = np.array(date_str_s)
        if date_str_s not in data_validation:
            
            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/Banca_Recolhas')
            
            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            # Process the data line by line, excluding the last line
            lines = file_data.readlines()
            data_rows = [process_data(row) for row in lines[:-1]]  # Exclude the last line
            
            # Remove BOM if present
            if data_rows and data_rows[0].startswith('\ufeff'):
                data_rows[0] = data_rows[0][1:]
            
            # Join the data_rows without the last line (to exclude the empty line) and convert to bytes
            csv_data = '\n'.join(data_rows[:-1]).encode('utf-8')
            
            # Decode the bytes data using utf-8-sig to handle the BOM
            decoded_data = csv_data.decode('utf-8-sig')
            
            if decoded_data.strip():  # Check if the decoded data is not empty
                # Create a DataFrame from the cleaned data rows
                df = pd.read_csv(io.StringIO(decoded_data), sep=';')

                # Rename the DataFrame columns based on the mapping
                df.rename(columns=column_mapping, inplace=True)

                # Insert the data into the database
                try:
                    cur = con.cursor()
                    # Replace "NaN" values with None (NULL) for database insertion
                    df = df.where(pd.notna(df), None)

                    # Generate placeholders for the query based on the number of columns
                    placeholders = ', '.join(['%s'] * len(df.columns))

                    # Construct the INSERT query with placeholders
                    insert_query = f"INSERT INTO Banca_2023 ({', '.join(df.columns)}) VALUES ({placeholders})"

                    # Execute the query with the DataFrame values as parameters
                    psycopg2.extras.execute_batch(cur, insert_query, df.values)
                    con.commit()

                    end_time = time.time()
                    print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")
                except (Exception, psycopg2.DatabaseError) as error:
                    print(f"Error inserting data from file '{file}': {error}")
                    con.rollback()

ftp.quit()
con.close()


Recolhas

In [10]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='postgres',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = '''CREATE TABLE IF NOT EXISTS Recolhas_2023 (
            "data_ficheiro" text,
            "Código Orgânico" text,
            "Descrição" text,
            "Dia" text,
            "Giro" text,
            "Tipo" text,
            "Ponto Recolha" text,
            "Nome Cliente" text,
            "Morada de recolha" text,
            "Código postal recolha" text,
            "Qntd de Objetos Recolhidos" text,
            "Estado" text,
            "Hora Início" text,
            "Hora Fim" text,
            "Hora Real" text
        )'''


criar_db(sql)



In [11]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)

cur.execute("SELECT DISTINCT  data_ficheiro as dias FROM Recolhas_2023  ORDER BY DIAS ASC")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

[]


In [12]:
import pandas as pd
import numpy as np
import io
import time
from ftplib import FTP
import psycopg2
import psycopg2.extras
import datetime


current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/Banca_Recolhas')

files = ftp.nlst()

def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')
    return decoded_line.strip()

data_validation = []

con = psycopg2.connect(host='localhost', 
                    database='postgres',
                    user='postgres', 
                    password='postgres')

cur = con.cursor()

cur.execute("SELECT DISTINCT  data_ficheiro as dias FROM Recolhas_2023  ORDER BY DIAS ASC")

for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

#cur.close()

# List of column names
columns = ['"Código Orgânico"', '"Descrição"', '"Dia"', '"Giro"', '"Tipo"', '"Ponto Recolha"', '"Nome Cliente"',
           '"Morada de recolha"', '"Código postal recolha"', '"Qntd de Objetos Recolhidos"', '"Estado"', '"Hora Início"',
           '"Hora Fim"', '"Hora Real"']

# Assuming you have a 'files' list defined earlier
for file in files:
    start_time = time.time()
    if file.startswith("R"):
        date_str_s = file[19:27].replace("/", "")
        date_str_s_array = np.array(date_str_s)
        if date_str_s not in data_validation:

            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/Banca_Recolhas')

            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            # Process the data line by line, excluding the last line
            lines = file_data.readlines()
            data_rows = [process_data(row) for row in lines[:-1]]  # Exclude the last line

            # Remove BOM if present
            if data_rows and data_rows[0].startswith('\ufeff'):
                data_rows[0] = data_rows[0][1:]

            # Join the data_rows without the last line (to exclude the empty line) and convert to bytes
            csv_data = '\n'.join(data_rows[:-1]).encode('utf-8')

            # Decode the bytes data using utf-8-sig to handle the BOM
            decoded_data = csv_data.decode('utf-8-sig')

            if decoded_data.strip():  # Check if the decoded data is not empty
                # Create a DataFrame from the cleaned data rows, specifying column names
                df = pd.read_csv(io.StringIO(decoded_data), sep=';', names=columns)
                df = df.iloc[1:]

                # Add the 'data_ficheiro' column and fill it with date_str_s value
                df['data_ficheiro'] = date_str_s

                # Insert the data into the database
                try:
                    # Replace "NaN" values with None (NULL) for database insertion
                    df = df.where(pd.notna(df), None)

                    # Generate placeholders for the query based on the number of columns
                    placeholders = ', '.join(['%s'] * len(df.columns))

                    # Construct the INSERT query with placeholders
                    insert_query = f"INSERT INTO Recolhas_2023 ({', '.join(df.columns)}) VALUES ({placeholders})"

                    # Execute the query with the DataFrame values as parameters
                    psycopg2.extras.execute_batch(cur, insert_query, df.values)
                    con.commit()

                    end_time = time.time()
                    print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")
                except (Exception, psycopg2.DatabaseError) as error:
                    print(f"Error inserting data from file '{file}': {error}")
                    con.rollback()


ftp.quit()
con.close()


File 'RECSG4366_RECOLHAS_20221230_20230104000037.CSV' inserted into the database: 2.1019954681396484 seconds
File 'RECSG4366_RECOLHAS_20221231_20230104000037.CSV' inserted into the database: 4.710844993591309 seconds
File 'RECSG4366_RECOLHAS_20230101_20230104000037.CSV' inserted into the database: 1.6353471279144287 seconds
File 'RECSG4366_RECOLHAS_20230102_20230104000037.CSV' inserted into the database: 1.913191556930542 seconds
File 'RECSG4366_RECOLHAS_20230103_20230104010501.CSV' inserted into the database: 1.4153037071228027 seconds
File 'RECSG4366_RECOLHAS_20230104_20230106000030.CSV' inserted into the database: 2.5992321968078613 seconds
File 'RECSG4366_RECOLHAS_20230105_20230106010501.CSV' inserted into the database: 1.548105001449585 seconds
File 'RECSG4366_RECOLHAS_20230106_20230107010502.CSV' inserted into the database: 1.4656322002410889 seconds
File 'RECSG4366_RECOLHAS_20230107_20230108010504.CSV' inserted into the database: 1.5988492965698242 seconds
File 'RECSG4366_RECOLH

REDE BASE


In [13]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='postgres',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = ''' CREATE TABLE IF NOT EXISTS REDE_BASE_2023 (
                                    DATA_CRIACAO      text,
                                    CENTRO    text,
                                    Giro text,
                                    LOPTICA  text,
                                    JANELA_HORARIA text,
                                    NOME text,
                                    MORADA text,
                                    CP text,
                                    LOCALIDADE text,
                                    COD_T_EVEN text,
                                    DATA_EVENTO text,
                                    LATITUDE text,
                                    LONGITUDE text) '''
criar_db(sql)


In [14]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute("SELECT distinct  SUBSTRING(DATA_CRIACAO, 7, 4) ||  SUBSTRING(DATA_CRIACAO, 4, 2) ||  SUBSTRING(DATA_CRIACAO, 1, 2) as dias FROM REDE_BASE_2023 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

['20230102' '20230103' '20230104' '20230105' '20230106' '20230107'
 '20230109' '20230110' '20230111' '20230112' '20230113' '20230114'
 '20230116' '20230117' '20230118' '20230119' '20230120' '20230121'
 '20230122' '20230123' '20230124' '20230126' '20230127' '20230128'
 '20230130' '20230131' '20230201' '20230202' '20230203' '20230204'
 '20230206' '20230207' '20230208' '20230209' '20230210' '20230211'
 '20230213' '20230214' '20230215' '20230216' '20230217' '20230218'
 '20230220' '20230221' '20230222' '20230223' '20230224' '20230225'
 '20230226' '20230227' '20230228' '20230301' '20230302' '20230303'
 '20230304' '20230306' '20230307' '20230309' '20230310' '20230311'
 '20230313' '20230314' '20230315' '20230316' '20230317' '20230318'
 '20230320' '20230321' '20230322' '20230323' '20230324' '20230325'
 '20230327' '20230328' '20230329' '20230330' '20230331' '20230401'
 '20230402' '20230403' '20230404' '20230405' '20230406' '20230407'
 '20230410' '20230411' '20230412' '20230413' '20230414' '20230

In [15]:
from ftplib import FTP
import os
import io
import datetime
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import time

current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/EventosPDA_Rede_base')

files = ftp.nlst()

def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')
    return decoded_line.strip()

data_validation = []


cur = con.cursor()

cur.execute("SELECT distinct  SUBSTRING(DATA_CRIACAO, 7, 4) ||  SUBSTRING(DATA_CRIACAO, 4, 2) ||  SUBSTRING(DATA_CRIACAO, 1, 2) as dias FROM REDE_BASE_2023 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

cur.close()

for file in files:
    start_time = time.time()
    if file.startswith("E"):
        date_str_s = file[14:22].replace("/", "")
        month= file[18:20].replace("/", "")
        date_str_s= np.array(date_str_s)
        if date_str_s not in data_validation and month == "12":
            print(date_str_s)
            
            
            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/EventosPDA_Rede_base')
            
            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            # Process the data line by line, excluding the last line
            lines = file_data.readlines()
            data_rows = [process_data(row) for row in lines[:-1]]  # Exclude the last line
            
            try:
                df = pd.read_csv(io.StringIO('\n'.join(data_rows)), sep=';')
            except pd.errors.ParserError as pe:
                print(f"ParserError occurred while reading CSV in file '{file}':")
                print(pe)
                continue  
            #df = pd.read_csv(io.StringIO('\n'.join(data_rows)), sep=';')
            
            ftp.quit()
            
            if len(data_rows)==1: 
                continue
            else:
                # Create a DataFrame from the cleaned data rows


                # Insert the data into the database
                for index, row in df.iterrows():
                    try:
                        cur = con.cursor()
                        # Replace "NaN" values with None (NULL) for database insertion
                        row = [None if pd.isna(value) else value for value in row]
                        placeholders = ', '.join(['%s'] * len(row))
                        insert_query = f"INSERT INTO REDE_BASE_2023 ({', '.join(df.columns)}) VALUES ({placeholders})"
                        cur.execute(insert_query, tuple(row))
                    except (Exception, psycopg2.DatabaseError) as error:
                        print(f"Error inserting row {index+1}: {error}")
                        con.rollback()
                    else:
                        con.commit()
                        
                end_time = time.time()    
                print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")


con.close()




20221231
File 'EVRBP4348_ENV_20221231_20230101010501.CSV' inserted into the database: 2.031161069869995 seconds
20231201
File 'EVRBP4348_ENV_20231201_20231202010502.CSV' inserted into the database: 88.51515483856201 seconds
20231202
File 'EVRBP4348_ENV_20231202_20231203010503.CSV' inserted into the database: 91.62183356285095 seconds
20231203
20231204
File 'EVRBP4348_ENV_20231204_20231205010500.CSV' inserted into the database: 395.0610146522522 seconds
20231205
File 'EVRBP4348_ENV_20231205_20231206010501.CSV' inserted into the database: 303.18179965019226 seconds
20231206
File 'EVRBP4348_ENV_20231206_20231207010502.CSV' inserted into the database: 354.85433530807495 seconds
20231207
File 'EVRBP4348_ENV_20231207_20231208010501.CSV' inserted into the database: 326.85896825790405 seconds
20231208
File 'EVRBP4348_ENV_20231208_20231209010501.CSV' inserted into the database: 69.08388781547546 seconds
20231209
File 'EVRBP4348_ENV_20231209_20231210010500.CSV' inserted into the database: 73.254

lista de tarefas

In [16]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='postgres',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = ''' CREATE TABLE IF NOT EXISTS Lista_tarefas_2023 (
                                    LISTA_TAREFAS      text,
                                    GIRO    text,
                                    CENTRO text,
                                    COLABORADOR  text,
                                    CODIGO_SERVICO text,
                                    DATA_HORA_ENTRADA_LT text,
                                    TIPO_SERVICO text,
                                    QUANTIDADE text,
                                    NOME text,
                                    ESTADO_SERVICO text,
                                    DATA_SERVICO text,
                                    DATA_FECHO_LT text,
                                    EVENTO text,
                                    RAZAO_SITUACAO text) '''
criar_db(sql)


In [17]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute("SELECT distinct  SUBSTRING(LISTA_TAREFAS, 1, 8) as dias FROM Lista_tarefas_2023 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

['20230509' '20230510' '20230511' '20230512' '20230515' '20230516'
 '20230517' '20230518' '20230519' '20230521' '20230522' '20230523'
 '20230524' '20230525' '20230526' '20230527' '20230528' '20230529'
 '20230530' '20230531' '20230601' '20230603' '20230604' '20230605'
 '20230606' '20230607' '20230608' '20230609' '20230612' '20230613'
 '20230614' '20230615' '20230616' '20230618' '20230619' '20230620'
 '20230621' '20230622' '20230623' '20230624' '20230626' '20230627'
 '20230628' '20230629' '20230630' '20230701' '20230703' '20230704'
 '20230705' '20230706' '20230707' '20230710' '20230711' '20230712'
 '20230713' '20230714' '20230717' '20230718' '20230719' '20230720'
 '20230721' '20230722' '20230724' '20230725' '20230726' '20230727'
 '20230728' '20230729' '20230731' '20230801' '20230802' '20230803'
 '20230804' '20230805' '20230806' '20230807' '20230808' '20230809'
 '20230810' '20230811' '20230814' '20230815' '20230816' '20230817'
 '20230818' '20230820' '20230821' '20230822' '20230823' '20230

In [18]:
from ftplib import FTP
import os
import io
import datetime
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import time

current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/Listas_Tarefas')

files = ftp.nlst()


def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')
    return decoded_line.strip()

data_validation = []


cur = con.cursor()

cur.execute("SELECT distinct  SUBSTRING(LISTA_TAREFAS, 1, 8) as dias FROM Lista_tarefas_2023 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

cur.close()

for file in files:
    
    start_time = time.time()
    if file.startswith("L"):
        Date_str_s = file[11:19].replace("/", "")
        month= file[15:17].replace("/", "")
        date_str_s= np.array(Date_str_s)
        if date_str_s not in data_validation and month == "12":        
            print(date_str_s)
            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/Listas_Tarefas')
            
            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            # Process the data line by line, excluding the last line
            lines = file_data.readlines()
            data_rows = [process_data(row) for row in lines[:-1]]  # Exclude the last line
            
            try:
                df = pd.read_csv(io.StringIO('\n'.join(data_rows)), sep=';')
            except pd.errors.ParserError as pe:
                print(f"ParserError occurred while reading CSV in file '{file}':")
                print(pe)
                continue  
            #df = pd.read_csv(io.StringIO('\n'.join(data_rows)), sep=';')
            
            ftp.quit()
            
            if len(data_rows)==1: 
                continue
            else:
                # Create a DataFrame from the cleaned data rows


                # Insert the data into the database
                for index, row in df.iterrows():
                    try:
                        cur = con.cursor()
                        # Replace "NaN" values with None (NULL) for database insertion
                        row = [None if pd.isna(value) else value for value in row]
                        placeholders = ', '.join(['%s'] * len(row))
                        insert_query = f"INSERT INTO Lista_tarefas_2023 ({', '.join(df.columns)}) VALUES ({placeholders})"
                        cur.execute(insert_query, tuple(row))
                    except (Exception, psycopg2.DatabaseError) as error:
                        print(f"Error inserting row {index+1}: {error}")
                        con.rollback()
                    else:
                        con.commit()
                        
                end_time = time.time()    
                print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")


con.close()


20231201
File 'LT4991_ENV_20231201_20231202010002.CSV' inserted into the database: 123.68473672866821 seconds
20231202
File 'LT4991_ENV_20231202_20231203010003.CSV' inserted into the database: 103.91985583305359 seconds
20231203
File 'LT4991_ENV_20231203_20231204010001.CSV' inserted into the database: 7.006869077682495 seconds
20231204
File 'LT4991_ENV_20231204_20231205010000.CSV' inserted into the database: 584.328590631485 seconds
20231205
File 'LT4991_ENV_20231205_20231206010001.CSV' inserted into the database: 517.2238373756409 seconds
20231206
File 'LT4991_ENV_20231206_20231207010002.CSV' inserted into the database: 494.7058012485504 seconds
20231207
File 'LT4991_ENV_20231207_20231208010001.CSV' inserted into the database: 420.3752353191376 seconds
20231208
File 'LT4991_ENV_20231208_20231209010000.CSV' inserted into the database: 132.6025321483612 seconds
20231209
File 'LT4991_ENV_20231209_20231210010000.CSV' inserted into the database: 102.16893243789673 seconds
20231210
File 'LT